<a href="https://colab.research.google.com/github/CamK2/ComputerVision/blob/main/CV_Mobilnet_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Learning for Object Identification

In this lab, we download and run a recent deeplearning model for computer vision that is optimized for minimal resources. In should be able to identify many common objects in everyday life.

The fundamental objective of this lab is to:
* under stand the big picture process of model training and the landscape of sharing models
* become familiar with obtaining and using a recent public deeplearning model
* understand some of the limitations of these models (especially minimal models optimized to run on limited hardware).

Transfer Learning - the process of training one network on a large broad dataset, then using this generic model along with additional specialized input to continue training to obtain a more specialized network.
Examples: AlexNet (old convolutional network) and MobilNet (optimized for small model size)

Imagenet - 


There are many deep learning models available for general object identification. However, as we ultimately want this model to run in colab as well as mobile computer hardware, it is crucial that the algorithm run without the enormous computational requirements typical of many models.  

We specifically use Google's [*MobileNets*](https://ai.googleblog.com/2017/06/mobilenets-open-source-models-for.html) family of computer vision models for tensorflow offered a reasonable balace between algorithm performance and hardware requirements.  

Google's MobileNetV2 is a neural-network-based **pretrained** model. MobileNetV2 was also designed to have a much smaller parameter count than typical computer vision models. 




We use Google's *tensorflow* open source software library for deeplearning.

In [1]:
# Here we install the tensorflow models into the colab environment  
!git clone https://github.com/tensorflow/models



Cloning into 'models'...
remote: Enumerating objects: 77758, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 77758 (delta 70), reused 114 (delta 62), pack-reused 77621
Receiving objects: 100% (77758/77758), 593.38 MiB | 26.35 MiB/s, done.
Resolving deltas: 100% (55246/55246), done.


In [2]:
# Here we download the mobilenet 2 model
from __future__ import print_function
from IPython import display 
base_name = checkpoint_name = 'mobilenet_v2_1.0_224' #@param
url = 'https://storage.googleapis.com/mobilenet_v2/checkpoints/' + checkpoint_name + '.tgz'
print('Downloading from ', url)
!wget {url}
print('Unpacking')
!tar -xvf {checkpoint_name}.tgz
checkpoint = checkpoint_name + '.ckpt'

display.clear_output()
print('Successfully downloaded checkpoint from ', url,
      '. It is available as', checkpoint)


Successfully downloaded checkpoint from  https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_224.tgz . It is available as mobilenet_v2_1.0_224.ckpt


In [5]:
# Here we install tf_slim, a lightweight library for  
# evaluating models in TensorFlow
import sys
sys.path.append('/content/models/research/slim')
!pip install tf_slim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 352 kB 5.1 MB/s 


In [12]:
!wget https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG -O panda.jpg

--2022-10-07 17:17:55--  https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116068 (113K) [image/jpeg]
Saving to: ‘panda.jpg’

panda.jpg           100%[===================>] 113.35K  --.-KB/s    in 0.03s   

2022-10-07 17:17:56 (3.67 MB/s) - ‘panda.jpg’ saved [116068/116068]



In [13]:
# load the trained model into tensorflow
import tensorflow.compat.v1 as tf
import tf_slim as slim
from nets.mobilenet import mobilenet_v2

tf.compat.v1.disable_eager_execution()
tf.reset_default_graph()

# For simplicity we just decode jpeg inside tensorflow.
# But one can provide any input obviously.
file_input = tf.placeholder(tf.string, ())

image = tf.image.decode_jpeg(tf.read_file(file_input))

images = tf.expand_dims(image, 0)
images = tf.cast(images, tf.float32) / 128.  - 1
images.set_shape((None, None, None, 3))
images = tf.image.resize_images(images, (224, 224))

# Note: arg_scope is optional for inference.
with slim.arg_scope(mobilenet_v2.training_scope(is_training=False)):
  logits, endpoints = mobilenet_v2.mobilenet(images)
  
# Restore using exponential moving average since it produces (1.5-2%) higher 
# accuracy
ema = tf.train.ExponentialMovingAverage(0.999)
vars = ema.variables_to_restore()

saver = tf.train.Saver(vars)  

## Test Images
Here we demonstrate processing of images to generate text labels for the target candidates.

In [16]:
# The wget command downloads images from the internet.
!wget "https://robbreport.com/wp-content/uploads/2022/01/Cielo2.jpg"
!wget 'https://www.defensenews.com/resizer/U2oS79JL6Z7cX9bXU9zUMBI9bJk=/1024x0/filters:format(jpg):quality(70)/cloudfront-us-east-1.images.arcpublishing.com/archetype/OIFRAG2XKFDPPKI5KVUURA5474' -O sideTank.jpg.jpg

--2022-10-07 17:21:52--  https://robbreport.com/wp-content/uploads/2022/01/Cielo2.jpg
Resolving robbreport.com (robbreport.com)... 192.0.66.24
Connecting to robbreport.com (robbreport.com)|192.0.66.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 880729 (860K) [image/jpeg]
Saving to: ‘Cielo2.jpg’

Cielo2.jpg          100%[===================>] 860.09K  --.-KB/s    in 0.07s   

2022-10-07 17:21:52 (12.5 MB/s) - ‘Cielo2.jpg’ saved [880729/880729]

--2022-10-07 17:21:53--  https://www.defensenews.com/resizer/U2oS79JL6Z7cX9bXU9zUMBI9bJk=/1024x0/filters:format(jpg):quality(70)/cloudfront-us-east-1.images.arcpublishing.com/archetype/OIFRAG2XKFDPPKI5KVUURA5474
Resolving www.defensenews.com (www.defensenews.com)... 23.213.26.137, 23.213.26.147, 2600:1408:5400:17::17dd:e3a9, ...
Connecting to www.defensenews.com (www.defensenews.com)|23.213.26.137|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2022-10-07 17:21:53 ERROR 403: Forbidden.



<img src='https://www.defensenews.com/resizer/U2oS79JL6Z7cX9bXU9zUMBI9bJk=/1024x0/filters:format(jpg):quality(70)/cloudfront-us-east-1.images.arcpublishing.com/archetype/OIFRAG2XKFDPPKI5KVUURA5474.jpg' width=300>

In [19]:
from IPython import display
import pylab
from datasets import imagenet
import PIL
#display.display(display.Image('sideTank.jpg'))

with tf.Session() as sess:
  saver.restore(sess,  checkpoint)
  x = endpoints['Predictions'].eval(feed_dict={file_input: 'Cielo2.jpg'})
label_map = imagenet.create_readable_names_for_imagenet_labels()  
print("Top 1 prediction: ", x.argmax(),label_map[x.argmax()], x.max())

Top 1 prediction:  484 castle 0.6609741


In [11]:
# add image where it fails